피쳐들 타입에 따른 전처리
- 범주형피쳐 : 명목형/순서형 =>인코딩
    - 명목형 => OneHotEncoding
    - 순서형 => OrdinalEncoding
- 수치형피쳐
    - 범주형 변환 ? : 데이터가 가진 의미 체크
    - 수치형 => 스케일링 여부

- 타겟 컬럼에 대한 처리
    - 범주형 => LabelEncoding
    - 수치형 => 그대로

- titanic.csv ,  auto_mpg.csv
- 전처리 함수화

In [69]:
## 모듈 로딩 
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import TargetEncoder
from sklearn.preprocessing import OrdinalEncoder


함수정의 <hr>

In [70]:
# ================================
# 함수이름 ; apply_label_encoder
# 함수기능 : 타깃 y에 LabelEncoder를 적용해 정수 라벨로 변환
# 매개변수 :
#   - y : 타깃 데이터 (Series 또는 1차원 배열 형태)
# 결과반환 :
#   - y_encoded : (n, 1) 형태의 2차원 numpy 배열 (정수 라벨)
# ================================
def apply_label_encoder(y):
    """y → (n,1) array + encoder 반환"""
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)
    return y_encoded.reshape(-1, 1)



# ================================
# 함수이름 ; apply_ordinal_encoder
# 함수기능 : 순서가 있는 범주형 컬럼들에 OrdinalEncoder를 적용해 정수 값으로 인코딩
# 매개변수 :
#   - df           : 원본 DataFrame
#   - ordinal_cols : Ordinal 인코딩을 적용할 컬럼명 리스트
# 결과반환 :
#   - arr : (n, len(ordinal_cols)) 형태의 2차원 numpy 배열
# ================================
def apply_ordinal_encoder(df, ordinal_cols):
    enc = OrdinalEncoder()
    temp = df[ordinal_cols].copy()
    for col in ordinal_cols:
        temp[col] = temp[col].fillna(temp[col].median())

    arr = enc.fit_transform(temp)
    return arr



# ================================
# 함수이름 ; apply_onehot_encoder
# 함수기능 : 순서가 없는 범주형 컬럼들에 OneHotEncoder를 적용해 더미(0/1) 벡터로 변환
# 매개변수 :
#   - df          : 원본 DataFrame
#   - onehot_cols : One-Hot 인코딩을 적용할 컬럼명 리스트
# 결과반환 :
#   - arr : (n, 인코딩된 전체 더미 컬럼 수) 형태의 2차원 numpy 배열
# ================================
def apply_onehot_encoder(df, onehot_cols):
    temp = df[onehot_cols].fillna("Missing")
    ohe = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
    arr = ohe.fit_transform(temp)
    return arr



# ================================
# 함수이름 ; apply_target_encoder
# 함수기능 : 다범주/문자열 범주형 컬럼들에 TargetEncoder를 적용해
#           각 카테고리를 타깃 평균값 기반의 수치로 변환
# 매개변수 :
#   - df          : 원본 DataFrame
#   - target_cols : Target 인코딩을 적용할 컬럼명 리스트
#   - y           : 타깃 데이터 (Series 또는 1차원 배열)
# 결과반환 :
#   - arr : (n, len(target_cols)) 형태의 2차원 numpy 배열
# ================================
def apply_target_encoder(df, target_cols, y):
    temp = df[target_cols].fillna("Missing").astype(str)
    te = TargetEncoder()
    arr = te.fit_transform(temp, y)
    return arr


##### [ 타이타닉DB ] <hr>

In [71]:
df = pd.read_csv('../Data/titanic.csv')

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   survived     891 non-null    int64  
 1   pclass       891 non-null    int64  
 2   sex          891 non-null    object 
 3   age          714 non-null    float64
 4   sibsp        891 non-null    int64  
 5   parch        891 non-null    int64  
 6   fare         891 non-null    float64
 7   embarked     889 non-null    object 
 8   class        891 non-null    object 
 9   who          891 non-null    object 
 10  adult_male   891 non-null    bool   
 11  deck         203 non-null    object 
 12  embark_town  889 non-null    object 
 13  alive        891 non-null    object 
 14  alone        891 non-null    bool   
dtypes: bool(2), float64(2), int64(4), object(7)
memory usage: 92.4+ KB


,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


- 인코딩 x 컬럼
    - age   : 나이 (연속형 숫자)
    - sibsp : 형제/배우자 수(정수)
    - parch : 부모/자식 수 (정수)
    - fare : 운임 (연속형 숫자)

- OrdinalEncoder : 순서가 있는 범주형
    - pclass : 등석

- OneHotEncoder : 순서 없는, 카테고리 수가 적은 범주형
    - sex : 성별
    - embarked : 선착장

- TargetEncoder : 카테고리가 많아서 onehot이 부담스러움
    - deck : 객실

- LabelEncoder : 타겟에 사용
    - survived


In [72]:

# ===============================================================
#  실제 타이타닉 데이터 불러오기 + 인코딩 처리
# ===============================================================
print("=== 원본 데이터 컬럼 ===")
print(df.columns)
print(df.head())



=== 원본 데이터 컬럼 ===
Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'deck', 'embark_town',
       'alive', 'alone'],
      dtype='object')
   survived  pclass     sex   age  sibsp  parch     fare embarked  class  \
0         0       3    male  22.0      1      0   7.2500        S  Third   
1         1       1  female  38.0      1      0  71.2833        C  First   
2         1       3  female  26.0      0      0   7.9250        S  Third   
3         1       1  female  35.0      1      0  53.1000        S  First   
4         0       3    male  35.0      0      0   8.0500        S  Third   

     who  adult_male deck  embark_town alive  alone  
0    man        True  NaN  Southampton    no  False  
1  woman       False    C    Cherbourg   yes  False  
2  woman       False  NaN  Southampton   yes   True  
3  woman       False    C  Southampton   yes  False  
4    man        True  NaN  Southampton    no   True  


In [73]:

# 2) 타깃(y)와 피처(X) 분리
y = df["survived"]
X = df.drop(columns=["survived"])


In [74]:
# ===============================================================
# 2) 필요없는 컬럼 제거
# ===============================================================
drop_cols = ["class", "alive"]
X = X.drop(columns=drop_cols)

# bool → 0/1
X["adult_male"] = X["adult_male"].astype(int)
X["alone"] = X["alone"].astype(int)


# ===============================================================
# 3) 컬럼 분류
# ===============================================================
ordinal_cols = ["pclass"]
onehot_cols = ["sex", "embarked", "who", "embark_town"]
target_cols = ["deck"]
numeric_cols = ["age", "sibsp", "parch", "fare", "adult_male", "alone"]


# ===============================================================
# 4) 인코딩 수행 — numpy array 반환
# ===============================================================

ord_arr = apply_ordinal_encoder(X, ordinal_cols)
onehot_arr = apply_onehot_encoder(X, onehot_cols)
target_arr = apply_target_encoder(X, target_cols, y)

numeric_arr = X[numeric_cols].to_numpy()


# ===============================================================
# 5) concatenate 로 최종 X 만들기
# ===============================================================
X_final = np.concatenate([ord_arr, onehot_arr, target_arr, numeric_arr], axis=1)
display(X_final)

# ===============================================================
# 6) y 도 숫자형으로
# ===============================================================
y_arr = apply_label_encoder(y)

print("X_final shape:", X_final.shape)
print("y shape:", y_arr.shape)



array([[ 2.    ,  0.    ,  1.    , ...,  7.25  ,  1.    ,  0.    ],
       [ 0.    ,  1.    ,  0.    , ..., 71.2833,  0.    ,  0.    ],
       [ 2.    ,  1.    ,  0.    , ...,  7.925 ,  0.    ,  1.    ],
       ...,
       [ 2.    ,  1.    ,  0.    , ..., 23.45  ,  0.    ,  0.    ],
       [ 0.    ,  0.    ,  1.    , ..., 30.    ,  1.    ,  1.    ],
       [ 2.    ,  0.    ,  1.    , ...,  7.75  ,  1.    ,  1.    ]],
      shape=(891, 21))

X_final shape: (891, 21)
y shape: (891, 1)


#### [ auto_mpg DB ]<hr>

In [75]:
df = pd.read_csv('../Data/auto_mpg.csv')
df.info()
display(df.head())
display(df.columns)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           398 non-null    float64
 1   cylinders     398 non-null    int64  
 2   displacement  398 non-null    float64
 3   horsepower    398 non-null    object 
 4   weight        398 non-null    int64  
 5   acceleration  398 non-null    float64
 6   model year    398 non-null    int64  
 7   origin        398 non-null    int64  
 8   car name      398 non-null    object 
dtypes: float64(3), int64(4), object(2)
memory usage: 28.1+ KB


,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino


Index(['mpg', 'cylinders', 'displacement', 'horsepower', 'weight',
       'acceleration', 'model year', 'origin', 'car name'],
      dtype='object')

##### [인코딩 설정]
- 숫자 그대로
    - displacement : 배기량
    - horsepower : 마력
    - weight : 차량 무게
    - acceleration : 가속
    - model year : 연식
- OrdinalEncoder
    - cylinders : 기통
- TargetEncoder
    - car name : 이름


In [76]:
# y, X 분리
y = df['mpg']                   # 타겟
X = df.drop(columns=['mpg'])    # 피쳐

ordinal_cols  = ['cylinders']
onehot_cols   = ['origin']
target_cols   = ['car name']
numeric_cols  = ['displacement', 'horsepower', 'weight', 'acceleration', 'model year']

# 인코딩
ord_arr       = apply_ordinal_encoder(X, ordinal_cols)
onehot_arr    = apply_onehot_encoder(X, onehot_cols)
target_arr     = apply_target_encoder(X, target_cols, y)
numeric_arr            = X[numeric_cols].to_numpy()

X_final = np.concatenate(
    [ord_arr, onehot_arr, target_arr, numeric_arr],
    axis=1
)

display(X_final)

y_arr = apply_label_encoder(y)  


array([[4.0, 1.0, 0.0, ..., 3504, 12.0, 70],
       [4.0, 1.0, 0.0, ..., 3693, 11.5, 70],
       [4.0, 1.0, 0.0, ..., 3436, 11.0, 70],
       ...,
       [1.0, 1.0, 0.0, ..., 2295, 11.6, 82],
       [1.0, 1.0, 0.0, ..., 2625, 18.6, 82],
       [1.0, 1.0, 0.0, ..., 2720, 19.4, 82]],
      shape=(398, 10), dtype=object)